In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import numpy as np
import pandas as pd
import json
import pickle as cp
import matplotlib.pyplot as plt
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
import torch

In [3]:
def getScore(model,X):
    input = tf.convert_to_tensor(X)
    with tf.GradientTape() as tape:
      tape.watch(input)
      output = model(input)
    gradients = np.absolute(tape.gradient(output, input))

    g = gradients.sum(axis=1)[:,1:]

    init = 0
    monthly_gradients = []
    for days in mnth_days:
        mgrad = gradients[init:init+days]
        monthly_gradients.append(mgrad.sum(axis=1).sum(axis=0)/days)
        init=init+days
    monthly_gradients = np.array(monthly_gradients)

    return gradients ### Returning the raw gradients instead of monthly aggregated values (temporary)

In [4]:
def addSpanLag(data, events=[], look_back=7, look_ahead = 7, season=0,span=0):
    ### Adding span (required for weekly, monthly average data)
    if span>1:
        nts = []
        ind = 0
        while ind < len(ts):
            tmpts = ts[ind:ind+span]
            print (tmpts)
            dt = tmpts[0]
            vals = [k for k in tmpts]
            pr = sum(vals)/len(vals)
            nts.append((dt,pr))
            ind+=span
        ts = nts


    ### Adding lag (based on look back and look ahead)

    if len(events) > 0:
        input = np.append(data,events,axis=1)
    else:
        input = data
    if season>0:
        season = np.array([k%season for k in range(data.shape[0])])
        season = season.reshape(len(season),1)
        input = np.append(input,season,axis=1)


    ind = 0
    X_data = []
    y_data = []

    while ind + look_back + look_ahead < len(data):

        X = input[ind:ind+look_back]
        Y = data[ind+look_back+look_ahead]
        X_data.append(X)
        y_data.append(Y)
        ind+=1
    X_data = np.array(X_data)
    y_data = np.array(y_data)

    return X_data, y_data

In [5]:
#### TO run this block return monthly aggregated gradients from the getScore function

crops = ['Onion','Potato','Rice','Wheat']
path='/home/sunchak/projects/news_analytics/REN/'
event_dict = {}
for crop in crops:

  model = keras.models.load_model(path+'onedayahead_logreturn/ren_'+crop)
  mnth_days = [31,28,31,30,31,30,31,31,30,31,30,31]*3
  fileformat = path+'food_price_data/%s.csv'
  data = pd.read_csv(fileformat%(crop))
  price = data['price']
  dates = data['date']
  #foodpriceDf = pd.Series(food_price,index=dates)
  start_date = '2012-12-24'
  end_date = '2015-12-31'
  price.index = dates
  range_price = price[start_date:end_date]

  food_price = np.log(range_price).diff().dropna()
  price_diff = np.array(food_price).reshape(food_price.shape[0],1)


  train = np.load(path+'embeddings/events_train.npy')
  test = np.load(path+'embeddings/events_test.npy')
  events = np.append(train,test, axis=0)[2548:2548+1102]

  X, y   = addSpanLag(price_diff,events,look_back=7,look_ahead=0,season=0)
  gradients = getScore(model,X)
  #gradients.shape
  evnt_grads = gradients[:,1:]
  top1 = []
  top10 = []
  for k in range(36):
    a = evnt_grads[k]
    ind = np.argpartition(a, -10)[-10:]
    top10.append(np.flip(ind))
    top1.append(np.argmax(a))
  event_dict[crop] = {}
  for evnt in top1:
    event_dict[crop][evnt] = evnt_grads[:,evnt]
cp.dump(event_dict,open(os.path.join(path,'event importance','event_scores.dict'),'wb'))

In [52]:
#### TO run this block return raw gradients from the getScore function. This block is computing the product of event values and gradients for each day jan 2013

crops = ['Onion','Potato','Wheat','Rice']
path='/home/sunchak/projects/news_analytics/REN/'
crop = 'Onion'
mnth_days = [31,28,31,30,31,30,31,31,30,31,30,31]*3
fileformat = path+'food_price_data/%s.csv'
for crop in crops:
  print(crop)
  model_path = 'onedayahead_logreturn/ren_'+crop
  event_dict = {}

  #if True:

  model = keras.models.load_model(path+model_path)
  
  data = pd.read_csv(fileformat%(crop))
  price = data['price']
  dates = data['date']
  #foodpriceDf = pd.Series(food_price,index=dates)
  start_date = '2012-12-25'
  end_date = '2015-12-31'
  price.index = dates
  price = np.log(price).diff()[1:]
  range_price = price[2548:2548+1102]#[start_date:end_date]


  #food_price = np.log(range_price).diff()[1:]
  price_diff = np.array(range_price).reshape(range_price.shape[0],1)


  train = np.load(path+'embeddings/events_train.npy')
  test = np.load(path+'embeddings/events_test.npy')
  events = np.append(train,test, axis=0)[2549:2549+1102]



  X, y   = addSpanLag(price_diff,events,look_back=7,look_ahead=0,season=0)
  gradients = getScore(model,X)
  gradients = gradients[:,:,1:]
  events_lagged = X[:,:,1:]
  #print(X.shape,events_lagged.shape)
  #c=np.multiply(a[2557:,b)

  st = 0
  top_events = []
  for i in range(36):
    
    end = st+mnth_days[i]
    #print(st,end)
    
    c = np.multiply(gradients[st:end],events_lagged[st:end])
    #print(c.shape)
    c = (c.sum(axis=1).sum(axis=0)/30).reshape(c.shape[2],1)
    
    top = np.argmax(c)
    topval = np.max(c,axis=1)
    
    #for i in range(end-st):
    ind = np.argpartition(c, -10,axis=0)[-10:]
    top10 = np.flip(ind).flatten()
    #top10 = np.array(top10)
    top_events+=list(top10  )
    st = end
  
  top_events = set(top_events)

  event_vals_months = []
  st = 0
  for i in range(36):
    end = st+mnth_days[i]
    #print(st,end)
    
    c = np.multiply(gradients[st:end],events_lagged[st:end])
    #print(c.shape)
    c = (c.sum(axis=1).sum(axis=0)/30).reshape(c.shape[2],1)
    event_vals = []
    for k in top_events:
      event_vals.append(c[k])
    # mx = max(event_vals)
    # mn = min(event_vals)
    # event_vals=[(k-mn)/(mx-mn) for k in event_vals]
    event_vals_months.append(event_vals)

  event_vals_months = np.array(event_vals_months)
  event_vals_months = event_vals_months.reshape(event_vals_months.shape[0],event_vals_months.shape[1]).T
  print(event_vals_months.shape)


  
  fig, ax = plt.subplots()
  im = ax.imshow(event_vals_months,cmap=plt.colormaps["Reds"])
  
  


Onion
(31, 250)
[0.01570053 0.01482384 0.01468655 0.02056872 0.01812815 0.01642361
 0.0164947  0.01760472 0.01655908 0.01617377 0.01480369 0.01349703
 0.0162228  0.01387695 0.01494383 0.01359611 0.01327919 0.01374925
 0.01185074 0.0108838  0.01358006 0.01528514 0.01193547 0.01142083
 0.01271103 0.01594691 0.01718018 0.01085398 0.01549435 0.01232106
 0.01335017]
